# NAVM.jl 测试

✨Powered by [**IpynbCompile.jl**](https://github.com/ARCJ137442/IpynbCompile.jl)

## 模块导入

模块清单：

- JuNarsese
- NAVM
- Test

In [1]:
include("#import.jl")

JuNarsese v2.6.3


┌ Info: NAVM导入成功！
│   names(NAVM) =
│    34-element Vector{Symbol}:
│     Symbol("@nair")
│     Symbol("@nair_rule")
│     :Backend
│     :BackendModule
│     :CMD_CYC
│     ⋮
│     :source_type
│     :target_type
│     :transform
│     :try_transform
└     :tryparse_cmd
Precompiling NAVM_Implements
        Info Given NAVM_Implements was explicitly requested, output will be shown live 
JuNarsese v2.6.3
  ✓ NAVM_Implements
  1 dependency successfully precompiled in 5 seconds. 20 already precompiled.
  1 dependency had output during precompilation:
┌ NAVM_Implements
│  [Output was shown above]
└  


"all"

## 测试 / 前端

In [2]:
fea = FE_TextParser(StringParser_ascii)
fes = FE_TextParser(SExprParser)

@show transform(fea, "<A --> B>.")
@show fes([
    "(Inheritance (Word A) (Word B))"
    "12"
])

@show fea("/VOL 10")

transform(fea, "<A --> B>.") = NAIR_CMD[CMD_NSE(<A --> B>. %1.0;0.5%)]
fes(["(Inheritance (Word A) (Word B))"; "12"]) = NAIR_CMD[CMD_NSE(<A --> B>), CMD_CYC(12)]
fea("/VOL 10") = NAIR_CMD[CMD_VOL(10)]


1-element Vector{NAIR_CMD}:
 CMD_VOL(10)

## 测试 / 后端

In [3]:
ben = BE_OpenNARS()
beo = BE_ONA()
bep = BE_NARS_Python()
bey = BE_PyNARS()
bej = BE_OpenJunars()

for be in [ben, beo, bep, bey, bej]
    @info be be([
        NAVM.CMD_REM("这是一段不会被翻译的注释")
        NAVM.CMD_NSE(nse"<A --> B>.")
        NAVM.CMD_NSE(nse"<B --> C>.")
        NAVM.CMD_CYC(5)
        # NAVM.CMD_UNK(5, Operator, Int, :a123, r"123", nse"<A --> B>.")
    ])
end

┌ Info: BE_OpenNARS()
│   be([NAVM.CMD_REM("这是一段不会被翻译的注释"); NAVM.CMD_NSE(nse"<A --> B>."); NAVM.CMD_NSE(nse"<B --> C>."); NAVM.CMD_CYC(5)]) =
│    3-element Vector{String}:
│     "<A --> B>. %1.0;0.5%"
│     "<B --> C>. %1.0;0.5%"
└     "5"
┌ Info: BE_ONA()
│   be([NAVM.CMD_REM("这是一段不会被翻译的注释"); NAVM.CMD_NSE(nse"<A --> B>."); NAVM.CMD_NSE(nse"<B --> C>."); NAVM.CMD_CYC(5)]) =
│    3-element Vector{String}:
│     "<A --> B>. %1.0;0.5%"
│     "<B --> C>. %1.0;0.5%"
└     "5"
┌ Info: BE_NARS_Python()
│   be([NAVM.CMD_REM("这是一段不会被翻译的注释"); NAVM.CMD_NSE(nse"<A --> B>."); NAVM.CMD_NSE(nse"<B --> C>."); NAVM.CMD_CYC(5)]) =
│    2-element Vector{String}:
│     "(A --> B). %1.0;0.5%"
└     "(B --> C). %1.0;0.5%"
┌ Info: BE_PyNARS()
│   be([NAVM.CMD_REM("这是一段不会被翻译的注释"); NAVM.CMD_NSE(nse"<A --> B>."); NAVM.CMD_NSE(nse"<B --> C>."); NAVM.CMD_CYC(5)]) =
│    3-element Vector{String}:
│     "<A --> B>. %1.0;0.5%"
│     "<B --> C>. %1.0;0.5%"
└     "5"
┌ Info: BE_OpenJunars()
│   be([NAVM.CMD_REM("这是一段

# 测试 / 前后端协同

In [4]:
chain_f::Function = chain(fea, bej)
@show chain_f
@show chain_f([
    # 多行文本测试
    """
    # 下面输入一段基本的三段论推理
    # 基本逻辑：「A是B」「B是C」⇒「A是C」
    """
    "<A -->B>."
    "<B--> C>."
    "<A--> C>?"
    "50"
])

for be in [ben, beo, bep, bey, bej]
    @info "从「$(fes)字串」到「$(be)命令」"
    join(
        (be ∘ fes)("""
        /RES memory
        /LOA memory *
        # 下面输入一段基本的三段论推理
        # 基本逻辑：「A是B」「B是C」⇒「A是C」
        (SentenceJudgement (Inheritance (Word A) (Word B)) )
        (SentenceJudgement (Inheritance (Word B) (Word C)) )
        (SentenceQuestion  (Inheritance (Word A) (Word C)) )
        /VOL 10
        50
        /CYC 50
        /INF memory
        /SAV memory *
        """),
        "\n"
    ) |> println
end

chain_f = BE_OpenJunars() ∘ FE_TextParser(StringParser_ascii)
chain_f(["# 下面输入一段基本的三段论推理\n# 基本逻辑：「A是B」「B是C」⇒「A是C」\n"; "<A -->B>."; "<B--> C>."; "<A--> C>?"; "50"]) = ["<A --> B>. %1.0;0.5%", "<B --> C>. %1.0;0.5%", "<A --> C>?", ":c 50"]


[ Info: 从「FE_TextParser(SExprParser)字串」到「BE_OpenNARS()命令」
┌ Warning: BE_OpenNARS(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_OpenNARS(): 未知的指令「CMD_LOA("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
*volume=10
50
50


┌ Warning: BE_OpenNARS(): 未知的指令「CMD_INF("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_OpenNARS(): 未知的指令「CMD_SAV("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
[ Info: 从「FE_TextParser(SExprParser)字串」到「BE_ONA()命令」


*reset
*load
<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
*volume=10
50
50
*save


...

┌ Warning: BE_NARS_Python(): 未知的指令「CMD_VOL(10)」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
[ Info: 从「FE_TextParser(SExprParser)字串」到「BE_PyNARS()命令」
┌ Warning: BE_PyNARS(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
50
50


┌ Warning: BE_PyNARS(): 未知的指令「CMD_LOA("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_PyNARS(): 未知的指令「CMD_VOL(10)」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_PyNARS(): 未知的指令「CMD_INF("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_PyNARS(): 未知的指令「CMD_SAV("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
[ Info: 从「FE_TextParser(SExprParser)字串」到「BE_OpenJunars()命令」


<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
:c 50
:c 50
:p


┌ Warning: BE_OpenJunars(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_OpenJunars(): 未知的指令「CMD_LOA("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_OpenJunars(): 未知的指令「CMD_VOL(10)」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Warning: BE_OpenJunars(): 未知的指令「CMD_SAV("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


## 尝试自编译

In [5]:
# %ignore-cell
using IpynbCompile: compile_notebook
THIS_PATH = "runtests.ipynb"
TARGET_PATH = "runtests.jl"
compile_notebook(THIS_PATH, TARGET_PATH)

┌ Debug: Loading object cache file C:\Users\Administrator\.julia\compiled\v1.10\IpynbCompile\pIVMh_pEIZk_1.dll for IpynbCompile [4eb781bf-a71e-403a-9d46-9d48649f04b2]
└ @ Base loading.jl:1051


2262